In [1]:
!pip install stackstac

  Using cached stackstac-0.4.1-py3-none-any.whl (62 kB)


In [31]:
import os
from osgeo import gdal
import pystac
from pystac_client import Client
import rasterio as rio
import stackstac
import xarray as xr
from dask import distributed

In [32]:
dist_env = stackstac.DEFAULT_GDAL_ENV.updated(dict(
    GDAL_DISABLE_READDIR_ON_OPEN='TRUE',
    GDAL_HTTP_COOKIEFILE=os.path.expanduser('~/cookies.txt'),
    GDAL_HTTP_COOKIEJAR=os.path.expanduser('~/cookies.txt'))
)

In [5]:
cluster = distributed.LocalCluster()
client = distributed.Client(cluster)
client.dashboard_link

'http://127.0.0.1:8787/status'

In [6]:
STAC_URL = 'https://cmr.earthdata.nasa.gov/stac'

In [7]:
catalog = Client.open(f"{STAC_URL}/LPCLOUD")

In [8]:
search = catalog.search(
    collections = ['HLSL30.v2.0', 'HLSS30.v2.0'],
    intersects = {'type': 'Polygon',
                  'coordinates': [[[-101.67271614074707, 41.04754380304359],
                                   [-101.65344715118408, 41.04754380304359],
                                   [-101.65344715118408, 41.06213891056728],
                                   [-101.67271614074707, 41.06213891056728],
                                   [-101.67271614074707, 41.04754380304359]]]},
    datetime = '2021-05/2021-08'
)               

In [9]:
search.matched()

113

In [44]:
item_col = search.get_all_items()
item_col

In [45]:
tile_item_col = [x for x in item_col if 'T13TGF' in x.id]

In [46]:
item_collection = pystac.ItemCollection(items=tile_item_col)

In [47]:
item_collection

In [48]:
data = stackstac.stack(item_collection, 
                       assets=['B04', 'B02'], 
                       epsg=32613, 
                       resolution=30,
                       gdal_env=dist_env)

In [49]:
data

,Array,Chunk
Bytes,13.01 GiB,8.00 MiB
Shape,"(57, 2, 3911, 3915)","(1, 1, 1024, 1024)"
Count,2052 Tasks,1824 Chunks
Type,float64,numpy.ndarray


In [50]:
data.sel(band='B04').isel(time=[0])

<xarray.DataArray 'stackstac-616c3920d64d98e3874f76c6822be2c1' (time: 1,
                                                                y: 3911, x: 3915)>
dask.array<getitem, shape=(1, 3911, 3915), dtype=float64, chunksize=(1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates:
  * time            (time) datetime64[ns] 2021-05-04T17:30:13.428000
    id              (time) <U34 'HLS.L30.T13TGF.2021124T173013.v2.0'
    band            <U3 'B04'
  * x               (x) float64 6.97e+05 6.97e+05 ... 8.144e+05 8.144e+05
  * y               (y) float64 4.604e+06 4.604e+06 ... 4.487e+06 4.487e+06
    eo:cloud_cover  (time) int64 36
    end_datetime    (time) <U24 '2021-05-04T17:30:37.319Z'
    start_datetime  (time) <U24 '2021-05-04T17:30:13.428Z'
    epsg            int64 32613
Attributes:
    spec:        RasterSpec(epsg=32613, bounds=(696990, 4486590, 814440, 4603...
    crs:         epsg:32613
    transform:   | 30.00, 0.00, 696990.00|\n| 0.00,-30.00, 4603920.00|\n| 0.0...
    resolution:  30